The purpose of this document is to create a database of fake company employees and their personal information, including the health care plan to which they are subscribed.

In [1]:
import psycopg

We need to randomly generate some junk data. It needs to include basic employee information such as first and last name, email address, and health care plan. The health care plan is important because it will be used to refine the vector query made by an agent. I may also include an "is manager" binary field to help refine queries for the HR policy vector database. Some documentation may be reserved for managers.

In [2]:
first_names = ['Adam', 'Susanna', 'Jack', 'Charles', 'Alice']
last_names = ['Krull', 'Krull', 'Craig', 'Patterson', 'Wonderland']
emails = ['ak@company.ai', 'sk@company.ai', 'jc@company.ai', 'cp@company.ai', 'aw@company.ai']
plans = ['Bronze', 'Gold', 'Gold', 'Bronze', 'Silver']
is_manager = [False, True, False, True, False]

We need to massage it into a list of tuples for insertion into a SQL table.

In [19]:
entries = []

for i in range(len(first_names)):
    new_tup = (first_names[i], last_names[i], emails[i], plans[i], is_manager[i])
    entries.append(new_tup)

print(entries)    

[('Adam', 'Krull', 'ak@company.ai', 'Bronze', False), ('Susanna', 'Krull', 'sk@company.ai', 'Gold', True), ('Jack', 'Craig', 'jc@company.ai', 'Gold', False), ('Charles', 'Patterson', 'cp@company.ai', 'Bronze', True), ('Alice', 'Wonderland', 'aw@company.ai', 'Silver', False)]


By default, Postgres creates a database of name "postgres". We'll connect to it and create a new table in the database for our employees. I'm using the same connection information from the other documents to create the connection to the hosted postgres database on my PC.

In [28]:
connection = 'user=langchain password=langchain host=localhost port=32768 dbname=postgres'

with psycopg.connect(connection) as conn:
    conn.set_autocommit(True)
    with conn.cursor() as cur:
        cur.execute('drop table if exists employees')
        cur.execute("""
            CREATE TABLE employees (
                    id serial primary key,
                    first_name varchar(20),
                    last_name varchar(20),
                    email varchar(30),
                    plan varchar(20),
                    is_manager boolean)
                """)

Great! We have a table. Let's insert the values from above.

In [29]:
with psycopg.connect(connection) as conn:
    with conn.cursor() as cur:
        cur.executemany(f"""
            INSERT into employees (first_name, last_name, email, plan, is_manager)
            VALUES (%s, %s, %s, %s, %s)""",
            entries)

Our database has been populated. Let's try a query to ensure things work as expected.

In [15]:
connection = 'user=langchain password=langchain host=localhost port=32768 dbname=postgres'

with psycopg.connect(connection) as conn:
    with conn.cursor() as cur:
        cur.execute("SELECT * FROM employees WHERE last_name = 'Krull'")
        result = cur.fetchone()
        print(result)

(1, 'Adam', 'Krull', 'ak@company.ai', 'Bronze', False)


Note about this query: I had to wrap the SQL statement in double quotes so my last name could be passed in single quotes. Otherwise, the database tried to interpet my last name as a column name.